In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [4]:
import pandas as pd
import investpy

### Data Intake

In [8]:
tickers = list(pd.read_csv("ind_nifty200list.csv")["Symbol"])

In [11]:
data = pd.read_csv("data.csv").set_index("Datetime")